<a href="https://colab.research.google.com/github/ajipurnama25/TUGBES_IMAGE-RETRIEVAL_TKCA_122_131/blob/main/tugas6_CBIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d denizkavi1/brain-tumor

100% 699M/700M [00:06<00:00, 155MB/s]
100% 700M/700M [00:06<00:00, 118MB/s]


In [4]:
!unzip brain-tumor.zip

Archive:  brain-tumor.zip
  inflating: 1/2300.png              
  inflating: 1/2301.png              
  inflating: 1/2302.png              
  inflating: 1/2303.png              
  inflating: 1/2304.png              
  inflating: 1/2305.png              
  inflating: 1/2306.png              
  inflating: 1/2307.png              
  inflating: 1/2308.png              
  inflating: 1/2309.png              
  inflating: 1/2310.png              
  inflating: 1/2311.png              
  inflating: 1/2312.png              
  inflating: 1/2313.png              
  inflating: 1/2314.png              
  inflating: 1/2315.png              
  inflating: 1/2316.png              
  inflating: 1/2317.png              
  inflating: 1/2318.png              
  inflating: 1/2319.png              
  inflating: 1/2320.png              
  inflating: 1/2321.png              
  inflating: 1/2322.png              
  inflating: 1/2323.png              
  inflating: 1/2324.png              
  inflating: 1/2325.png 

In [5]:
!mkdir '/content/brain_tumor'

In [6]:
!mv "/content/1" "/content/brain_tumor"
!mv "/content/2" "/content/brain_tumor"
!mv "/content/3" "/content/brain_tumor"

In [7]:
!pip install split-folders

In [8]:
!mkdir braintumor

In [9]:
import splitfolders

base_dir = "/content/brain_tumor"
splitfolders.ratio(base_dir, output="braintumor", 
                   ratio=(.8, .1, .1))

Copying files: 3064 files [00:02, 1107.87 files/s]


In [10]:
from imutils import paths
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

imagePathsTrain = list(paths.list_images("/content/braintumor/train"))
trainX = []
trainY = []

for imagePath in imagePathsTrain:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	trainX.append(image)
	trainY.append(label)

trainX = np.array(trainX, dtype="float32")
trainY = np.array(trainY)

lb = LabelEncoder()
trainY = lb.fit_transform(trainY)
trainY = to_categorical(trainY)

In [11]:
imagePathsTest = list(paths.list_images("/content/braintumor/test"))
testX = []
testY = []

for imagePath in imagePathsTest:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	testX.append(image)
	testY.append(label)

testX = np.array(testX, dtype="float32")
testY = np.array(testY)

lb = LabelEncoder()
testY = lb.fit_transform(testY)
testY = to_categorical(testY)

In [12]:
imagePathsTest = list(paths.list_images("/content/braintumor/val"))
valX = []
valY = []

for imagePath in imagePathsTest:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	valX.append(image)
	valY.append(label)

valX = np.array(valX, dtype="float32")
valY = np.array(valY)

lb = LabelEncoder()
valY = lb.fit_transform(valY)
valY = to_categorical(valY)

In [13]:
print("SHAPE OF TRAIN FEATURES: "+str(trainX.shape))
print("SHAPE OF TRAIN LABELS: "+str(trainY.shape))
print("SHAPE OF TEST FEATURES: "+str(testX.shape))
print("SHAPE OF TEST LABELS: "+str(testY.shape))
print("SHAPE OF VALIDATION FEATURES: "+str(valX.shape))
print("SHAPE OF VALIDATION LABELS: "+str(valY.shape))

SHAPE OF TRAIN FEATURES: (2450, 120, 120, 3)
SHAPE OF TRAIN LABELS: (2450, 3)
SHAPE OF TEST FEATURES: (309, 120, 120, 3)
SHAPE OF TEST LABELS: (309, 3)
SHAPE OF VALIDATION FEATURES: (305, 120, 120, 3)
SHAPE OF VALIDATION LABELS: (305, 3)


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [15]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [27]:
# Tulis Kode Anda Disini
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, Normalization, Dropout, BatchNormalization

In [25]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))
base_model.trainable = False ## Not trainable weights

58900480/58889256 [==============================] - 1s 0us/step


In [36]:

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = True

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [37]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(valX, valY),
	validation_steps=len(valX) // BS,
	epochs=EPOCHS)

Epoch 1/20
76/76 [==============================] - 69s 301ms/step - loss: 0.3731 - accuracy: 0.8532 - val_loss: 0.6521 - val_accuracy: 0.8984
Epoch 2/20
76/76 [==============================] - 22s 285ms/step - loss: 0.2517 - accuracy: 0.9178 - val_loss: 0.3874 - val_accuracy: 0.9115
Epoch 3/20
76/76 [==============================] - 22s 291ms/step - loss: 0.1930 - accuracy: 0.9342 - val_loss: 0.1385 - val_accuracy: 0.9508
Epoch 4/20
76/76 [==============================] - 22s 284ms/step - loss: 0.1602 - accuracy: 0.9446 - val_loss: 0.1615 - val_accuracy: 0.9541
Epoch 5/20
76/76 [==============================] - 22s 284ms/step - loss: 0.1647 - accuracy: 0.9479 - val_loss: 0.6073 - val_accuracy: 0.8656
Epoch 6/20
76/76 [==============================] - 22s 284ms/step - loss: 0.1090 - accuracy: 0.9603 - val_loss: 0.1454 - val_accuracy: 0.9672
Epoch 7/20
76/76 [==============================] - 22s 284ms/step - loss: 0.1047 - accuracy: 0.9648 - val_loss: 0.3356 - val_accuracy: 0.9344

In [38]:
from sklearn.metrics import classification_report

y_pred = np.around(model.predict(testX))
print(classification_report(testY, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97        72
           1       0.99      0.98      0.98       144
           2       0.97      0.97      0.97        93

   micro avg       0.98      0.97      0.97       309
   macro avg       0.98      0.97      0.97       309
weighted avg       0.98      0.97      0.97       309
 samples avg       0.97      0.97      0.97       309



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
